In [ ]:
!pip install --upgrade pip
!pip install transformers[sentencepiece]
!pip install -Uqq auto-gptq
!pip install pdfminer.six
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 7.3 MB/s 

# Model import

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig


In [ ]:
!GITHUB_ACTIONS=true

In [ ]:
model_name_or_path = "TheBloke/Luna-AI-Llama2-Uncensored-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=False,
        quantize_config=None)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# Embeddings

In [ ]:
from pdfminer.high_level import extract_text
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [ ]:
# Extracting text from PDF documents

def extract(fname, window_size, step_size):
    text = extract_text(fname)
    text = " ".join(text.split())
    text_tokens = text.split()

    sentences = []
    for i in range(0, len(text_tokens), step_size):
        window = text_tokens[i: i+window_size]
        if len(window) < window_size:
            break
        sentences.append(window)

    paragraphs = [" ".join(s) for s in sentences]
    print(paragraphs)
    return paragraphs

In [ ]:
# Embedding the extracted text paragraphs

def embed(paragraphs):

    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    model.max_seq_length = 512

    cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

    embeddings = model.encode(
        paragraphs,
        show_progress_bar=True,
        convert_to_tensor=True
    )

    return model, cross_encoder, embeddings

In [ ]:
# semantic search of text embeddings

def search(query, model, cross_encoder, embeddings, paragraphs, top_k):

    query_embeddings = model.encode(query, convert_to_tensor=True)
    query_embeddings = query_embeddings.cuda()
    hits = util.semantic_search(
        query_embeddings,
        embeddings,
        top_k=top_k,
    )[0]

    cross_input = [[query, paragraphs[hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_input)

    for idx in range(len(cross_scores)):
        hits[idx]["cross_score"] = cross_scores[idx]

    results = []
    hits = sorted(hits, key=lambda x: x["cross_score"], reverse=True)

    for hit in hits[:5]:
        results.append(paragraphs[hit["corpus_id"]].replace("\n", " "))

    return results

# Arguments

In [ ]:
FNAME = "text.pdf"
TOP_K = 32
WINDOW_SIZE = 128
STEP_SIZE = 100

In [ ]:
paragraphs = extract(
    FNAME,
    WINDOW_SIZE,
    STEP_SIZE
)

embedding_model, cross_encoder, embeddings = embed(paragraphs)

['HAN 09-ch02-039-082-9780123814791 2011/6/1 3:15 Page 39 #1 Getting to Know Your Data 2 It’s tempting to jump straight into mining, but ﬁrst, we need to get the data ready. This involves having a closer look at attributes and data values. Real-world data are typically noisy, enormous in volume (often several gigabytes or more), and may originate from a hodge- podge of heterogenous sources. This chapter is about getting familiar with your data. Knowledge about your data is useful for data preprocessing (see Chapter 3), the ﬁrst major task of the data mining process. You will want to know the following: What are the types of attributes or ﬁelds that make up your data? What kind of values does each attribute have? Which attributes are discrete, and which are', 'What are the types of attributes or ﬁelds that make up your data? What kind of values does each attribute have? Which attributes are discrete, and which are continuous-valued? What do the data look like? How are the values distrib

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
prompt = "what are attributes"

context = search(
    prompt,
    embedding_model,
    cross_encoder,
    embeddings,
    paragraphs,
    TOP_K,
)

prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to answer all of the users queries. Restrict your answer only to the given context.

USER: {prompt}
CONTEXT: {context}
ASSISTANT:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to answer all of the users queries. Restrict your answer only to the given context.

USER: what are attributes
CONTEXT: ['by attributes. Attributes can be nominal, binary, ordinal, or numeric. The values of a nominal (or categorical) attribute are symbols or names of things, where each value represents some kind of category, code, or state. Binary attributes are nominal attributes with only two possible states (such as 1 and 0 or true and false). If the two states are equally important, the attribute is symmetric; otherwise it is asymmetric. An ordinal attribute is an attribute with possible values that have a meaningful order or ranking among them, but the magnitude between successive values is not known. A numeric attribute is quantitative (i.e., it is a measurable quantity) represented in integer or real values. Numeric attribute types can be interval-scal